In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

## READING DATA

In [ ]:
df=pd.read_csv('/content/dataset.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')


In [ ]:
df.head()

In [ ]:
# prompt: get the different values in trafficcondition column

df['TrafficConditionNearby'].unique()

## ENCODING TRAFFIC CONDITION ------> LOW=1, AVG=2, HIGH=3

In [ ]:
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].str.lower().map({
    'low': 1, 'average': 2, 'high': 3
})

In [ ]:
df.head()

# BASELINE MODEL

In [ ]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema1(pw.Schema):
    SystemCodeNumber:str  #this is the id of the parking lots
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location


In [ ]:
class Model1Calculator:
  def __init__(self,base=10.0,alpha=1.0):
    self.base = base
    self.alpha = alpha
    self.prices = {}  #this is a dictionary which will contain the last price of parking a vehicle lot_id wise

  def update_price(self,lot_id,occupancy,capacity):
    if lot_id not in self.prices:
      self.prices[lot_id] = self.base

    occ_ratio=0
    if capacity>0:
      occ_ratio=occupancy/capacity
    new_price = self.prices[lot_id] + self.alpha * occ_ratio
    self.prices[lot_id]=new_price
    return round(min(new_price, 50), 2)

In [ ]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[['SystemCodeNumber',"Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

In [ ]:
df1=pd.read_csv('/content/parking_stream.csv')
df1.tail()

In [ ]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("/content/parking_stream.csv", schema=ParkingSchema1, input_rate=1000)

# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt)
)


In [ ]:
# Instantiate the calculator
model = Model1Calculator(alpha=0.05)

# Define UDF correctly
@pw.udf
def compute_price(system_code: str, occ: int, cap: int, t) -> float:
    return model.update_price(system_code, occ, cap)

# Apply it
price_stream = data_with_time.with_columns(
    price = compute_price(
        pw.this.SystemCodeNumber,
        pw.this.Occupancy,
        pw.this.Capacity,
        pw.this.t
    )
)


In [ ]:
# Step 8: Panel + Bokeh Plot
import panel as pn
pn.extension()
from bokeh.palettes import Category10, Category20
import pandas as pd
# Step 8: Panel + Bokeh Plot
import panel as pn
pn.extension()

def price_plot(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Model 1: Real-Time Parking Prices",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="green", legend_label="Price")
    fig.scatter("t", "price", source=source, size=6, color="red")
    fig.legend.location = "top_left"
    return fig

viz = price_stream.plot(price_plot, sorting_col="t")

# Show it in the notebook
# pn.panel(viz).show()
pn.Column(viz).servable()

viz = price_stream.plot(price_plot, sorting_col="t")

# Show it in the notebook
# pn.panel(viz).show()
pn.Column(viz).servable()


In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

# 2ND MODEL BASED ON MULTIPLE FACTORS

In [ ]:
class Model2Calculator:
    def __init__(self, base_price=10.0, alpha=1.0, beta=0.5, gamma=0.3, delta=1.0, epsilon=1.0, lambda_=0.5):
        self.base_price = base_price
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta
        self.epsilon = epsilon
        self.lambda_ = lambda_

    def get_vehicle_weight(self, vehicle_type: str) -> float:
        return {
            "bike": 0.5,
            "car": 1.0,
            "truck": 1.5
        }.get(vehicle_type.lower(), 1.0)

    def compute_demand(self, occupancy, capacity, queue, traffic, special_day, vehicle_type):
        occupancy_ratio = occupancy / capacity if capacity > 0 else 0
        vehicle_weight = self.get_vehicle_weight(vehicle_type)
        demand = (
            self.alpha * occupancy_ratio +
            self.beta * queue -
            self.gamma * traffic +
            self.delta * special_day +
            self.epsilon * vehicle_weight
        )
        return demand

    def update_price(self, occupancy, capacity, queue, traffic, special_day, vehicle_type):
        demand = self.compute_demand(occupancy, capacity, queue, traffic, special_day, vehicle_type)

        # Normalize demand into [0, 1] using sigmoid-like function or simple min-max
        normalized_demand = max(0, min(1, demand / 10))  # crude normalization

        price = self.base_price * (1 + self.lambda_ * normalized_demand)
        price = max(5.0, min(20.0, price))  # clip price
        return round(price, 2)


In [ ]:
# Instantiate model
model2 = Model2Calculator()

@pw.udf
def compute_model2_price(occupancy: int, capacity: int, queue: int, traffic: float,
                         special_day: int, vehicle_type: str) -> float:
    return model2.update_price(occupancy, capacity, queue, traffic, special_day, vehicle_type)


In [ ]:
class ParkingSchemaModel2(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: float
    IsSpecialDay: int
    VehicleType: str


In [ ]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[['SystemCodeNumber',"Timestamp", "Occupancy", "Capacity",'QueueLength','TrafficConditionNearby','IsSpecialDay','VehicleType']].to_csv("parking_stream_model2.csv", index=False)

In [ ]:
df2=pd.read_csv('/content/parking_stream_model2.csv')
df2.head()

In [ ]:
data1 = pw.demo.replay_csv("/content/parking_stream_model2.csv", schema=ParkingSchemaModel2, input_rate=50)
fmt1 = "%Y-%m-%d %H:%M:%S"

data_with_time1 = data1.with_columns(
    t = data1.Timestamp.dt.strptime(fmt)
)

price_stream1 = data_with_time1.with_columns(
    price = compute_model2_price(
        pw.this.Occupancy,
        pw.this.Capacity,
        pw.this.QueueLength,
        pw.this.TrafficConditionNearby,
        pw.this.IsSpecialDay,
        pw.this.VehicleType
    )
)
def price_plot(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Model 1: Real-Time Parking Prices",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="green", legend_label="Price")
    fig.scatter("t", "price", source=source, size=6, color="red")
    fig.legend.location = "top_left"
    return fig

viz1 = price_stream1.plot(price_plot, sorting_col="t")
pn.extension()
pn.Column(viz1).servable()


In [ ]:
# viz1 = price_stream1.plot(price_plot, sorting_col="t")
# pn.Column(viz1).servable()
%%capture --no-display
pw.run()